<a href="https://colab.research.google.com/github/GabeAspir/Patent-Prior-Art-Finder/blob/main/2nd%20Stage%20Development/WithCitationsEdited.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from gensim.models import Word2Vec
from _DevNLTKPatentPriorArtFinder import _DevNLTKPatentPriorArtFinder as paf
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
#Will read the .json to a dataframe
# Use the comment if you're not using Colab, and rather have it saved on your comp 
# jsonObject1= pd.io.json.read_json("JsonTTPatentsWithcitations.json")
jsonObject1 = pd.io.json.read_json('/content/5000PatentsAbstractAndCitationsJson.json')
jsonObject1

,Publication_Number,Abstract,Citations
0,US-2019086938-A1,An unmanned aerial vehicle (UAV) uses a first ...,"US-2015379874-A1,US-2016028471-A1,US-201633077..."
1,US-2019153590-A1,"Devices having an air bearing surface (ABS), t...","US-2014376352-A1,US-2015179194-A1,US-9129620-B..."
2,US-2016323114-A1,Temporal key generation devices and methods ar...,"US-2004000587-A1,US-2004144840-A1,US-200425514..."
3,US-2020010766-A1,The present invention provides apparatuses and...,"US-7462725-B2,US-7462725-B2"
4,US-2019373828-A1,A flow-through oxygen infuser which is an asse...,"US-1728951-A,US-2880549-A,US-3518831-A,US-3640..."
...,...,...,...
4995,US-2015125446-A1,The present invention is directed to the combi...,
4996,US-2016068817-A1,The present invention relates to drug discover...,
4997,US-2016213693-A1,T-lymphocyte invasion and metastases (Tiam-1) ...,
4998,US-2017042780-A1,The present invention provides an oral care co...,


In [7]:
myPaf = paf()
dframe = myPaf.init(jsonObject1,"Publication_Number", "Abstract")
dframe

,Publication_Number,Abstract,Citations,Tokens,BagOfWords,TF-IDF
0,US-2019086938-A1,An unmanned aerial vehicle (UAV) uses a first ...,"US-2015379874-A1,US-2016028471-A1,US-201633077...","[unmanned, aerial, vehicle, uav, uses, first, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,US-2019153590-A1,"Devices having an air bearing surface (ABS), t...","US-2014376352-A1,US-2015179194-A1,US-9129620-B...","[devices, air, bearing, surface, abs, device, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,US-2016323114-A1,Temporal key generation devices and methods ar...,"US-2004000587-A1,US-2004144840-A1,US-200425514...","[temporal, key, generation, devices, methods, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,US-2020010766-A1,The present invention provides apparatuses and...,"US-7462725-B2,US-7462725-B2","[present, invention, provides, apparatuses, pr...","[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0690793761055259, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,US-2019373828-A1,A flow-through oxygen infuser which is an asse...,"US-1728951-A,US-2880549-A,US-3518831-A,US-3640...","[flowthrough, oxygen, infuser, assemblage, one...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...
4995,US-2015125446-A1,The present invention is directed to the combi...,,"[present, invention, directed, combination, th...","[4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.22491385472791914, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4996,US-2016068817-A1,The present invention relates to drug discover...,,"[present, invention, relates, drug, discovery,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4997,US-2016213693-A1,T-lymphocyte invasion and metastases (Tiam-1) ...,,"[tlymphocyte, invasion, metastases, tiam, _NUM...","[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.11496546016879328, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4998,US-2017042780-A1,The present invention provides an oral care co...,,"[present, invention, provides, oral, care, com...","[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.298207422606011, 0.0, 0.0, 0.0, 0.0, 0.0, 0..."


I guess some patents have zero citations, unless we did something wrong again... 

In [8]:
#First model is created with the tokens in sentences
sentences= dframe['Tokens']
model = Word2Vec(sentences)

In [9]:
#Just an example to show that it's working
print(model.wv.most_similar(positive=['device']))

[('electronic', 0.9521160125732422), ('external', 0.9078956246376038), ('wireless', 0.8842828273773193), ('interface', 0.8766387701034546), ('terminal', 0.8694185018539429), ('station', 0.8691755533218384), ('twoway', 0.8614294528961182), ('receiving', 0.859796404838562), ('router', 0.8418717384338379), ('mobile', 0.8334463834762573)]


In [14]:
#Tokenizing all the citations and displaying it's header/column in the next codeblock
#Currently gets rid of doubles when you make it a set in the return statement
def tokenizeCitations(citationsString):
  string1 = citationsString.replace(',',' ')
  tokenized = word_tokenize(string1)
  return list(set(tokenized))

def allTokenizeCitations(dataframe, column_name):
  dataframe['TokenizedCitations'] = dataframe[column_name].apply(tokenizeCitations)

In [15]:
allTokenizeCitations(dframe, 'Citations')
dframe

,Publication_Number,Abstract,Citations,Tokens,BagOfWords,TF-IDF,TokenizedCitations
0,US-2019086938-A1,An unmanned aerial vehicle (UAV) uses a first ...,"US-2015379874-A1,US-2016028471-A1,US-201633077...","[unmanned, aerial, vehicle, uav, uses, first, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[US-2015379874-A1, US-2016028471-A1, US-201633..."
1,US-2019153590-A1,"Devices having an air bearing surface (ABS), t...","US-2014376352-A1,US-2015179194-A1,US-9129620-B...","[devices, air, bearing, surface, abs, device, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[US-2016275979-A1, US-9129620-B2, US-10014011-..."
2,US-2016323114-A1,Temporal key generation devices and methods ar...,"US-2004000587-A1,US-2004144840-A1,US-200425514...","[temporal, key, generation, devices, methods, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[US-2004255147-A1, US-2004000587-A1, US-976255..."
3,US-2020010766-A1,The present invention provides apparatuses and...,"US-7462725-B2,US-7462725-B2","[present, invention, provides, apparatuses, pr...","[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0690793761055259, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[US-7462725-B2]
4,US-2019373828-A1,A flow-through oxygen infuser which is an asse...,"US-1728951-A,US-2880549-A,US-3518831-A,US-3640...","[flowthrough, oxygen, infuser, assemblage, one...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[US-4148155-A, US-3640516-A, US-4315381-A, US-..."
...,...,...,...,...,...,...,...
4995,US-2015125446-A1,The present invention is directed to the combi...,,"[present, invention, directed, combination, th...","[4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.22491385472791914, 0.0, 0.0, 0.0, 0.0, 0.0,...",[]
4996,US-2016068817-A1,The present invention relates to drug discover...,,"[present, invention, relates, drug, discovery,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",[]
4997,US-2016213693-A1,T-lymphocyte invasion and metastases (Tiam-1) ...,,"[tlymphocyte, invasion, metastases, tiam, _NUM...","[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.11496546016879328, 0.0, 0.0, 0.0, 0.0, 0.0,...",[]
4998,US-2017042780-A1,The present invention provides an oral care co...,,"[present, invention, provides, oral, care, com...","[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.298207422606011, 0.0, 0.0, 0.0, 0.0, 0.0, 0...",[]


In [16]:
#Showing that both sentences and sentences2 are tokenized correctly
sentences

0       [unmanned, aerial, vehicle, uav, uses, first, ...
1       [devices, air, bearing, surface, abs, device, ...
2       [temporal, key, generation, devices, methods, ...
3       [present, invention, provides, apparatuses, pr...
4       [flowthrough, oxygen, infuser, assemblage, one...
                              ...                        
4995    [present, invention, directed, combination, th...
4996    [present, invention, relates, drug, discovery,...
4997    [tlymphocyte, invasion, metastases, tiam, _NUM...
4998    [present, invention, provides, oral, care, com...
4999    [present, disclosure, field, plant, breeding, ...
Name: Tokens, Length: 5000, dtype: object

In [17]:
sentences2 = dframe['TokenizedCitations']
sentences2

0       [US-2015379874-A1, US-2016028471-A1, US-201633...
1       [US-2016275979-A1, US-9129620-B2, US-10014011-...
2       [US-2004255147-A1, US-2004000587-A1, US-976255...
3                                         [US-7462725-B2]
4       [US-4148155-A, US-3640516-A, US-4315381-A, US-...
                              ...                        
4995                                                   []
4996                                                   []
4997                                                   []
4998                                                   []
4999                                                   []
Name: TokenizedCitations, Length: 5000, dtype: object

In [19]:
# Created the 2nd model based on the citations, and an example
model2 = Word2Vec(sentences2, min_count=1)
model2.wv.most_similar(positive=['US-2015379874-A1'])

[('US-9980756-B2', 0.3828250467777252),
 ('US-2014149041-A1', 0.37845855951309204),
 ('US-2018262529-A1', 0.36357924342155457),
 ('US-3576067-A', 0.34629717469215393),
 ('US-2851234-A', 0.3460381329059601),
 ('US-2019066509-A1', 0.3408134877681732),
 ('US-7302221-B2', 0.3401264250278473),
 ('US-2002057892-A1', 0.3400003910064697),
 ('US-2018310745-A1', 0.33463671803474426),
 ('US-5777306-A', 0.3313838839530945)]

In [20]:
#Finally the code for all patents that we have, to combine both the abstract and the citations in one document into a document Vector.
#Disclaimer - hasnt been tested yet

vecs= []

for (doc,citationList) in zip(sentences,sentences2):
  sum=np.empty(100)
  for word in doc:
    try:
      sum += model.wv[word]
    except:
      pass
  for citation in citationList:
    try:
      sum += model2.wv[citation]
    except:
      pass
  vecs.append(sum)

In [22]:
def vecCos(vectors, names):
    newTable = pd.DataFrame(cosine_similarity(vectors))
    newTable.columns = names
    newTable.index = names

    return newTable
vecTable = vecCos(vecs,dframe['Publication_Number'].tolist()) #,frame['PublicationNumber'].tolist()
vecTable

,US-2019086938-A1,US-2019153590-A1,US-2016323114-A1,US-2020010766-A1,US-2019373828-A1,US-2018349197-A1,US-2017175025-A1,US-2017227892-A1,US-2017271031-A1,US-2018105778-A1,US-2018226596-A1,US-2017338456-A1,US-2017095285-A1,US-2015277685-A1,US-2018276098-A1,US-2018241041-A1,US-2019252486-A1,US-2016281628-A1,US-2019184949-A1,US-2019017416-A1,US-2019075214-A1,US-2019004351-A1,US-2019334593-A1,US-2018316061-A1,US-2018137916-A1,US-2015292636-A1,US-2018350494-A1,US-2020079473-A1,US-2016036756-A1,US-2015318732-A1,US-2015140933-A1,US-2016296210-A1,US-2016213730-A1,US-2020225917-A1,US-2018322695-A1,US-2017346974-A1,US-2017327192-A1,US-2020028320-A1,US-2018344627-A1,US-2020073168-A1,...,US-2016060643-A1,US-2017009287-A1,US-2018218278-A1,US-2017137320-A1,US-2019151263-A1,US-2012148550-A1,US-2019167564-A1,US-2017022944-A1,US-2014330327-A1,US-2017364364-A1,US-2016218404-A1,US-2018157225-A1,US-2017357906-A1,US-2017088532-A1,US-2017103304-A1,US-2016250496-A1,US-2017016008-A1,US-2018002703-A1,US-2017002363-A1,US-2013272970-A1,US-2017203245-A1,US-2018126004-A1,US-2013296703-A1,US-2015225465-A1,US-2016263109-A1,US-2017166580-A1,US-2019074152-A1,US-2017265788-A1,US-2017119857-A1,US-2018098589-A1,US-2016065445-A1,US-2018123313-A1,US-2017064334-A1,US-2016267559-A1,US-2017141337-A1,US-2015125446-A1,US-2016068817-A1,US-2016213693-A1,US-2017042780-A1,US-2017051302-A1
US-2019086938-A1,1.000000,0.625083,0.942896,0.453005,0.444146,0.840072,0.210596,0.609212,0.923606,0.606482,0.508789,0.646947,0.568641,0.901844,0.810435,0.465047,0.651566,0.605945,0.776415,0.586527,0.876026,0.574999,0.947794,0.646048,0.852564,0.540460,0.422074,0.389889,0.745279,0.866764,0.855928,0.835654,0.264303,0.791292,0.756772,0.789348,0.628283,0.431853,0.397640,0.726532,...,0.396664,0.665218,0.835645,0.517345,0.233696,0.511215,0.488254,0.522730,0.666851,0.953847,0.546808,0.811498,0.805727,0.337045,0.904574,0.516205,0.626837,0.554893,0.515253,0.410700,0.371692,0.474985,0.758950,0.422305,0.342736,0.529566,0.690003,0.658298,0.482791,0.471659,0.853560,0.544016,0.803857,0.809970,0.505587,0.330259,0.396489,0.416020,0.219361,0.417935
US-2019153590-A1,0.625083,1.000000,0.735670,0.877531,0.876848,0.702804,0.838749,0.902294,0.564674,0.813840,0.896339,0.907206,0.866308,0.753887,0.695931,0.844844,0.853254,0.655435,0.833404,0.866183,0.823045,0.774247,0.728740,0.940629,0.716817,0.866137,0.783072,0.874912,0.532059,0.779582,0.721839,0.685764,0.743741,0.719863,0.852178,0.599213,0.756499,0.798490,0.902231,0.769259,...,0.773571,0.837601,0.630938,0.942566,0.707683,0.816014,0.886883,0.744544,0.904893,0.676025,0.933471,0.740752,0.707986,0.761226,0.666201,0.896543,0.812960,0.853069,0.870872,0.739840,0.910260,0.772959,0.865963,0.742252,0.786336,0.864920,0.806123,0.891852,0.857949,0.963985,0.605034,0.967133,0.718433,0.671960,0.867192,0.787580,0.765224,0.818011,0.719546,0.706354
US-2016323114-A1,0.942896,0.735670,1.000000,0.624412,0.608694,0.934295,0.401532,0.679270,0.933480,0.801098,0.552437,0.698057,0.544680,0.961358,0.929680,0.491780,0.664988,0.652920,0.822987,0.580489,0.916399,0.565009,0.985237,0.774988,0.949541,0.555441,0.378539,0.411235,0.863276,0.872622,0.953734,0.896256,0.460130,0.920775,0.873167,0.794213,0.560602,0.395873,0.559240,0.708644,...,0.492355,0.846663,0.941419,0.606817,0.474566,0.715376,0.680413,0.478595,0.683988,0.978030,0.606770,0.939696,0.939950,0.551818,0.912281,0.564644,0.816415,0.713463,0.679686,0.621808,0.503061,0.691766,0.843959,0.644847,0.541980,0.737438,0.637413,0.834400,0.672379,0.569108,0.932418,0.657408,0.912400,0.927501,0.529986,0.484987,0.616089,0.614952,0.380150,0.653753
US-2020010766-A1,0.453005,0.877531,0.624412,1.000000,0.962837,0.607782,0.892770,0.716620,0.435388,0.902504,0.696845,0.778612,0.635078,0.610166,0.639741,0.757170,0.548434,0.748611,0.801944,0.686799,0.607724,0.441458,0.600500,0.881311,0.659842,0.783225,0.628543,0.698427,0.543227,0.622520,0.685269,0.655828,0.930894,0.663369,0.753885,0.576187,0.573248,0.545995,0.965635,0.439162,...,0.865792,0.863900,0.607322,0.74

Looks like the first and third patents are pretty similar to each other, lets check it out.

In [23]:
print(dframe.at[0,'Abstract'])
print(dframe.at[0,'Citations'])
print(dframe.at[2,'Abstract'])
print(dframe.at[2,'Citations'])

An unmanned aerial vehicle (UAV) uses a first baseband processor to establish a first communication link with a ground network cell and a second baseband processor that establishes a second communication link with a user device. The second baseband processor is communicatively coupled to the first baseband processor such that the user device exchanges communication data with the core network via the first communication link and the second communication link. Flight-control hardware steers the UAV along a flight trajectory that is determined by a ground-based UAV controller based at least on a geolocation of the user device. The second baseband processor establishes the second communication link with the user device while the first baseband processor maintains the first communication link with the ground network cell.
US-2015379874-A1,US-2016028471-A1,US-2016330771-A1,US-10020872-B2,US-2015379874-A1,US-2016028471-A1,US-2016330771-A1,US-10020872-B2
Temporal key generation devices and met

Some of the citations are the same! That's great news. The abstracts don't seem too similar though, hmmm.

I'm comfortable saying the model works though.